<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [9]:
# !pip3 install folium==0.5.0 # DOESN'T WORK ANYMORE
# !pip3 install folium==0.6.0 # DIDN'T WORK EITHER, HAD TO INSTALL PYTHON OUTSIDE ANACONDA AND FOLIUM / MousePosition WORKED AGAIN
# !pip3 install wget

In [10]:
import folium
import wget
import pandas as pd

In [11]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [17]:
# Download and read the `spacex_launch_geo.csv`
# spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_csv_file = 'spacex_launch_geo.csv'
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [19]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [22]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [24]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [32]:
# Initial the map
start_coordinate = [40,-100]
site_map = folium.Map(location=start_coordinate, zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
launch_sites_df.reset_index()
for index, site in launch_sites_df.iterrows():
    site_latlon = [site['Lat'], site['Long']]
    circle = folium.Circle(site_latlon, radius=100, color='#d35400', fill=True).add_child(folium.Popup(site['Launch Site']))
    # Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
        site_latlon,
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'],
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)
    
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [38]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [42]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [44]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

spacex_df.insert(len(spacex_df.columns), 'marker_color', 'blue')
spacex_df

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,blue
1,CCAFS LC-40,28.562302,-80.577356,0,blue
2,CCAFS LC-40,28.562302,-80.577356,0,blue
3,CCAFS LC-40,28.562302,-80.577356,0,blue
4,CCAFS LC-40,28.562302,-80.577356,0,blue
5,CCAFS LC-40,28.562302,-80.577356,0,blue
6,CCAFS LC-40,28.562302,-80.577356,0,blue
7,CCAFS LC-40,28.562302,-80.577356,0,blue
8,CCAFS LC-40,28.562302,-80.577356,0,blue
9,CCAFS LC-40,28.562302,-80.577356,0,blue


In [45]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [47]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    site_latlon = [record['Lat'], record['Long']]
    marker = folium.map.Marker(
        site_latlon,
        # Create an icon as a text label
        icon = folium.Icon(color='white', icon_color=record['marker_color'])
        )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [55]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [58]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [60]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

launch_site_lat = 28.563197
launch_site_lon = -80.576820
coastline_lat = 28.56377
coastline_lon = -80.568
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

distance_coastline

0.8639957832515675

In [61]:
launch_site_latlon = [launch_site_lat, launch_site_lon]
coastline_latlon = [coastline_lat, coastline_lon]
coordinates = [launch_site_latlon, coastline_latlon]

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [63]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

distance_marker = folium.Marker(
    coastline_latlon,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "Coast line: {:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [65]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [75]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

city_lat = 28.61331
city_lon = -80.80788
city_latlon = [city_lat, city_lon]
city_distance = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
to_city_coordinates = [launch_site_latlon, city_latlon]

city_marker = folium.Marker(
    city_latlon,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "City: {:10.2f} KM".format(city_distance),
        )
    )
site_map.add_child(city_marker)

lines=folium.PolyLine(locations=to_city_coordinates, weight=1)
site_map.add_child(lines)


In [76]:
# railroad marker

rail_lat = 28.5721
rail_lon = -80.58527

rail_latlon = [rail_lat, rail_lon]
rail_distance = calculate_distance(launch_site_lat, launch_site_lon, rail_lat, rail_lon)
to_rail_coordinates = [launch_site_latlon, rail_latlon]

rail_marker = folium.Marker(
    rail_latlon,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "Railroad: {:10.2f} KM".format(rail_distance),
        )
    )
site_map.add_child(rail_marker)

lines=folium.PolyLine(locations=to_rail_coordinates, weight=1)
site_map.add_child(lines)


In [77]:
# highway marker

highway_lat = 28.56352
highway_lon = -80.57081
highway_latlon = [highway_lat, highway_lon]
highway_distance = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
to_highway_coordinates = [launch_site_latlon, highway_latlon]

highway_marker = folium.Marker(
    highway_latlon,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "Highway: {:10.2f} KM".format(highway_distance),
        )
    )
site_map.add_child(highway_marker)

lines=folium.PolyLine(locations=to_highway_coordinates, weight=1)
site_map.add_child(lines)


*My TODO:* Doing the same for the West Cost launch site in California


In [79]:
# find coordinate of the closet coastline
# e.g.,: Lat: 34.63285  Lon: -120.61071 @WestCoast / California
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# all the variables with suffix "WC", which stands for "West Coast"

launch_site_lat_WC = 34.63285
launch_site_lon_WC = -120.61071
coastline_lat_WC = 34.63691
coastline_lon_WC = -120.62453
distance_coastline_WC = calculate_distance(launch_site_lat_WC, launch_site_lon_WC, coastline_lat_WC, coastline_lon_WC)

distance_coastline_WC

1.342993480450541

In [80]:
launch_site_latlon_WC = [launch_site_lat_WC, launch_site_lon_WC]
coastline_latlon_WC = [coastline_lat_WC, coastline_lon_WC]
coordinates_WC = [launch_site_latlon_WC, coastline_latlon_WC]

In [81]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

distance_marker_WC = folium.Marker(
    coastline_latlon_WC,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "Coastline: {:10.2f} KM".format(distance_coastline_WC),
        )
    )
site_map.add_child(distance_marker_WC)
site_map

In [82]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

lines_WC=folium.PolyLine(locations=coordinates_WC, weight=1)
site_map.add_child(lines_WC)
site_map

In [83]:
# @WEST COAST / CALIFORNIA launch site
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

city_lat_WC = 34.64846
city_lon_WC = -120.45822
city_latlon_WC = [city_lat_WC, city_lon_WC]
city_distance_WC = calculate_distance(launch_site_lat_WC, launch_site_lon_WC, city_lat_WC, city_lon_WC)
to_city_coordinates_WC = [launch_site_latlon_WC, city_latlon_WC]

city_marker_WC = folium.Marker(
    city_latlon_WC,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "City: {:10.2f} KM".format(city_distance_WC),
        )
    )
site_map.add_child(city_marker_WC)

lines=folium.PolyLine(locations=to_city_coordinates_WC, weight=1)
site_map.add_child(lines)


In [84]:
# @WEST COAST / CALIFORNIA launch site
# railroad marker

rail_lat_WC = 34.63659
rail_lon_WC = -120.62347
rail_latlon_WC = [rail_lat_WC, rail_lon_WC]
rail_distance_WC = calculate_distance(launch_site_lat_WC, launch_site_lon_WC, rail_lat_WC, rail_lon_WC)
to_rail_coordinates_WC = [launch_site_latlon_WC, rail_latlon_WC]

rail_marker_WC = folium.Marker(
    rail_latlon_WC,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "Railroad: {:10.2f} KM".format(rail_distance_WC),
        )
    )
site_map.add_child(rail_marker_WC)

lines=folium.PolyLine(locations=to_rail_coordinates_WC, weight=1)
site_map.add_child(lines)


In [85]:
# @WEST COAST / CALIFORNIA launch site
# highway marker

highway_lat_WC = 34.63198
highway_lon_WC = -120.61811
highway_latlon_WC = [highway_lat_WC, highway_lon_WC]
highway_distance_WC = calculate_distance(launch_site_lat_WC, launch_site_lon_WC, highway_lat_WC, highway_lon_WC)
to_highway_coordinates_WC = [launch_site_latlon_WC, highway_latlon_WC]

highway_marker_WC = folium.Marker(
    highway_latlon_WC,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "Highway: {:10.2f} KM".format(highway_distance_WC),
        )
    )
site_map.add_child(highway_marker_WC)

lines=folium.PolyLine(locations=to_highway_coordinates_WC, weight=1)
site_map.add_child(lines)


In [86]:
site_map

*Let's compare both sites*

In [88]:
# comparison between: 
#   - site #0, the original in the exercise, which is in Florida, in the East Coast, (data_EC)
#   - and site #1: in California, in the West Coast (data_WC)

# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
data_EC = ['CCAFS-SLC-40 @ Florida', distance_coastline, city_distance, rail_distance, highway_distance]
data_WC = ['VAFB-SLC-4E @ California', distance_coastline_WC, city_distance_WC, rail_distance_WC, highway_distance_WC]

comparison_df = pd.DataFrame([data_EC, data_WC], columns=['Launch site', 'D2 coast', 'D2 city', 'D2 rail', 'D2 highway'])

comparison_df

,Launch site,D2 coast,D2 city,D2 rail,D2 highway
0,CCAFS-SLC-40 @ Florida,0.863996,23.245548,1.289202,0.588227
1,VAFB-SLC-4E @ California,1.342993,14.062340,1.239666,0.684138


In [89]:
# southern distance of Florida site
# we calculate the distance between lat_lon of WC site and lat EC site with lon WC site
# this gives us an imaginary straight line between the WC site and an imaginary point to the south, 
#    whose longitude is the same of WC site but its latitude is the corresponding to the EC site

southern_distance = calculate_distance(launch_site_lat_WC, launch_site_lon_WC, launch_site_lat, launch_site_lon_WC)

southern_distance

675.126490951533

In [90]:
# distance of the Florida's site to the Equator
# we calculate the distance between lat_lon of EC site and lon EC site with lat 0 (the Equator)
equator_distance_EC = calculate_distance(launch_site_lat, launch_site_lon, 0, launch_site_lon)

equator_distance_EC

3177.079638814171

In [91]:
# distance of the California's site to the Equator
# we calculate the distance between lat_lon of WC site and lon WC site with lat 0 (the Equator)
equator_distance_WC = calculate_distance(launch_site_lat_WC, launch_site_lon_WC, 0, launch_site_lon_WC)

equator_distance_WC

3852.2061297657046

## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |
